In [732]:
import pandas as pd
import numpy as np

In [733]:
# read raw data
df_train = pd.read_csv('../data/BADS_WS1718_known.csv', sep=',', na_values=['?', 'not_reported'])
df_test = pd.read_csv('../data/BADS_WS1718_class.csv', sep=',', na_values=['?', 'not_reported'])

In [734]:
# just to prove pythons superiority compared to R
#for col in df_train.columns:
#    print(df_train.groupby(col)['order_item_id'].nunique())

In [735]:
#for col in df_test.columns:
#    print(df_test.groupby(col)['order_item_id'].nunique())

In [736]:
# Data Preprocessing

In [737]:
# 1. Drop order_item_id -> no value
df_train = df_train.drop(['order_item_id'], axis=1)
df_test = df_test.drop(['order_item_id'], axis=1)

In [738]:
# 2. Convert date columns to datetime format:  order_date, delivery-date, user_reg_date, user_dob

In [739]:
date_columns = ['order_date', 'delivery_date', 'user_reg_date', 'user_dob']
for date in date_columns:
    df_train[date] = pd.to_datetime(df_train[date], format="%Y-%m-%d")

In [740]:
for date in date_columns:
    df_test[date] = pd.to_datetime(df_test[date], format="%Y-%m-%d")

In [741]:
# 3. Calculate additional columns
# 3.1 Length of customership  (LOC) (order_date - user_reg_date )
df_train['LOC'] = (df_train.order_date - df_train.user_reg_date) / np.timedelta64(1, 'D')
df_test['LOC'] = (df_test.order_date - df_test.user_reg_date) / np.timedelta64(1, 'D')

In [742]:
#3.2 age of the customers df['age'] at the time of order (order_date - user_dob)
df_train['age'] = np.round((df_train.order_date - df_train.user_dob) / np.timedelta64(1, 'Y'))
df_test ['age'] = np.round((df_test.order_date - df_test.user_dob) / np.timedelta64(1, 'Y'))

In [743]:
#3.3 delivery-time (DT) (order_date - delivery_date)
df_train['DT'] = (df_train.delivery_date - df_train.order_date) / np.timedelta64(1, 'D')
df_test['DT'] = (df_test.delivery_date - df_test.order_date) / np.timedelta64(1, 'D')

In [744]:
#3.4 Number of past purchases (NPP)
past_purchases = df_train.user_id.value_counts()
df_train['npp'] = 0
npp = []
for elem in zip(df_train.user_id, df_train.npp):
    npp.append(past_purchases[elem[0]])
df_train.npp = npp

past_purchases = df_test.user_id.value_counts()
df_test['npp'] = 0
npp = []
for elem in zip(df_test.user_id, df_test.npp):
    npp.append(past_purchases[elem[0]])
df_test.npp = npp

In [745]:
# 3.5 Return rate of the customer
# Update: NEVER use the target variable in feature engineering because it messes up the whole model
# df_train.groupby('user_id')['return'].sum() --> I don't include it because it would not be independent from the npp column

In [746]:
# 3.6 split order_date, delivery_date into --> *_month, *year

In [747]:
df_train['order_month'], df_train['order_year'] = df_train.order_date.dt.month, df_train.order_date.dt.year 
df_test['order_month'], df_test['order_year'] = df_test.order_date.dt.month, df_test.order_date.dt.year
df_train['delivery_month'], df_train['delivery_year'] = df_train.delivery_date.dt.month, df_train.delivery_date.dt.year
df_test['delivery_month'], df_test['delivery_year'] = df_test.delivery_date.dt.month, df_train.delivery_date.dt.year

In [748]:
df_train.columns

Index(['order_date', 'delivery_date', 'item_id', 'item_size', 'item_color',
       'brand_id', 'item_price', 'user_id', 'user_title', 'user_dob',
       'user_state', 'user_reg_date', 'return', 'LOC', 'age', 'DT', 'npp',
       'order_month', 'order_year', 'delivery_month', 'delivery_year'],
      dtype='object')

In [749]:
# Drop now obsolete columns
df_train = df_train.drop(['order_date', 'delivery_date', 'user_dob', 'user_reg_date'], axis=1)
df_test =  df_test.drop(['order_date', 'delivery_date', 'user_dob', 'user_reg_date'], axis=1)

In [750]:
# # useful helper
# with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#     print(#whatever)

In [751]:
# 5. Strategy for columns with a lot of different class labels
# 5.1 item_id --> 3 Categories: popular item (bought >= 300 times), normal item (300 > x > 50), rare item (50 >= num_bought)
def categorize_item_id(x, item_id_counts):
    x = item_id_counts[x]
    
    if x > 300:
        x = 'popular'
    elif x > 50:
        x = 'normal'
    else:
        x = 'rare'
        
    return x

item_id_train_counts = df_train.item_id.value_counts()
item_id_test_counts = df_test.item_id.value_counts()

df_train['item_popularity'] = df_train.item_id.apply(lambda x: categorize_item_id(x, item_id_train_counts))
df_test['item_popularity'] = df_test.item_id.apply(lambda x: categorize_item_id(x, item_id_test_counts))

In [752]:
# 5.2 item_size --> join big letters and small letters into same group if the same (L + l); if count > 1000 --> keep, else --> other
def categorize_item_size(x, item_size_count):
        
    count = item_size_count[x]
    
    if count < 1000:
        x = 'other'
    return x

df_train.item_size = df_train.item_size.apply(lambda x: x.lower())
df_test.item_size = df_test.item_size.apply(lambda x: x.lower())

item_size_train_counts = df_train.item_size.value_counts()
item_size_test_counts = df_test.item_size.value_counts()

df_train.item_size = df_train.item_size.apply(lambda x: categorize_item_size(x, item_size_train_counts))
df_test.item_size = df_test.item_size.apply(lambda x: categorize_item_size(x, item_size_test_counts))

In [753]:
# 5.3 item_color --> if count > 2000 --> keep; else --> other
def categorize_item_color(x, item_color_count):
    
    count = item_color_count[x]
    
    if count < 2000:
       x = 'other'
    
    return x

# Convert NaNs to other
df_train.item_color = df_train.item_color.fillna('other')
df_test.item_color = df_test.item_color.fillna('other')

item_color_train_counts = df_train.item_color.value_counts()
item_color_test_counts = df_test.item_color.value_counts()

df_train.item_color = df_train.item_color.apply(lambda x: categorize_item_color(x, item_color_train_counts))
df_test.item_color = df_test.item_color.apply(lambda x: categorize_item_color(x, item_color_test_counts))

In [754]:
# 5.4 brand_id --> if count > 2000 : popular; 2000 >= count > 500 : normal; Rest --> rare
def categorize_brand_id(x, brand_id_count):
    
    count = brand_id_count[x]
    
    if count > 2000:
        x = 'popular'
    elif count > 500:
        x = 'normal'
    else:
        x = 'rare'
        
    return x

brand_id_train_counts = df_train.brand_id.value_counts()
brand_id_test_counts = df_test.brand_id.value_counts()

df_train['brand_popularity'] = df_train.brand_id.apply(lambda x: categorize_brand_id(x, brand_id_train_counts))
df_test['brand_popularity'] = df_test.brand_id.apply(lambda x: categorize_brand_id(x, brand_id_test_counts))

In [755]:
# 5.5 delivery_month --> make nas to 'other'
df_train.delivery_month = df_train.delivery_month.fillna('other')
df_test.delivery_month = df_test.delivery_month.fillna('other')

In [756]:
# 5.6 delivery_year --> make nas to other
df_train.delivery_year = df_train.delivery_year.fillna('other')
df_test.delivery_year = df_test.delivery_year.fillna('other')

In [757]:
# 5.6 user_id --> nothing really that one could do here, since we already got number of purchases

In [758]:
# Again drop now unnecessary columns
df_train = df_train.drop(['item_id', 'brand_id', 'user_id'], axis=1)
df_test =  df_test.drop(['item_id', 'brand_id', 'user_id'], axis=1)

In [759]:
### All columns are fine now in terms of data type and structure ###

### Check columns for plausability ###

# item_size                      | Train: Okay | Test: Okay
# item_color                     | Train: Okay | Test: Okay
# item_price                     --> Zero values, and two 999 values. Cant for sure be outliers --> stay; Rest okay
# user_title                     | Train: Okay | Test: Okay
# user_state                     | Train: Okay | Test: Okay
# LOC                            | Train: Okay | Test: Okay
# age                            | Train: needs work | Test: needs work --> Some are older than 100 and  smaller 15 !! --FIXED--
# DT                             | Train: work | Test: work --> bunch of negative delivery times   --FIXED--
# npp                            | Train: Okay | Test: Okay 
# order_month                    | Train: Okay | Test: Okay
# order_year                     | Train: Okay | Test: Okay
# delivery_month                 | Train: Okay | Test: Okay --> NaNs bei beiden --FIXED--
# delivery_year                  | Train: Okay | Test: Okay --> NaNs bei beiden  und 1990 (unsinnig) --FIXED--
# item_popularity                | Train: Okay | Test: Okay
# brand_popularity               | Train: Okay | Test: Okay

In [760]:
# 1. fix age column: Every age > 80 == 80; and every age < 18 == 18
def fix_age(x):
    if x > 80:
        x = 80
    elif x < 18:
        x = 18
    return x

# Fill NAs with median
median_train_age = df_train.age.median()
median_test_age = df_test.age.median()

df_train.age = df_train.age.fillna(median_train_age)
df_test.age = df_test.age.fillna(median_test_age)

df_train.age = df_train.age.apply(lambda x: fix_age(x))
df_test.age = df_test.age.apply(lambda x: fix_age(x))

In [761]:
# 2.fix DT column: negative values come from 1990 delivery time --> impute to median delivery_time (mean from positive values)
def fix_DT(x, mean_DT):
    if x < 0:
        x = mean_DT
    return x

median_train_DT = df_train.DT[df_train.DT >= 0].median()
median_test_DT = df_test.DT[df_test.DT >= 0].median()

#Fill NA DTs with median
df_train.DT = df_train.DT.fillna(median_train_DT)
df_test.DT = df_test.DT.fillna(median_test_DT)

df_train.DT = df_train.DT.apply(lambda x: fix_DT(x, median_train_DT))
df_test.DT = df_test.DT.apply(lambda x: fix_DT(x, median_test_DT))

In [762]:
# 3. fix delivery_year --> 1990 values will just be smoothed to 2012
def fix_delivery_year(x):
    if x == 'other':
        return x
    if x < 2012:
        x = 2012
    return x

df_train.delivery_year = df_train.delivery_year.apply(lambda x: fix_delivery_year(x))
df_test.delivery_year = df_test.delivery_year.apply(lambda x: fix_delivery_year(x))

In [763]:
#### Data is correct and clean ###

In [767]:
df_train.columns

Index(['item_size', 'item_color', 'item_price', 'user_title', 'user_state',
       'return', 'LOC', 'age', 'DT', 'npp', 'order_month', 'order_year',
       'delivery_month', 'delivery_year', 'item_popularity',
       'brand_popularity'],
      dtype='object')

In [777]:
# test = pd.get_dummies(ord_test)
# test.columns
ord_test = pd.DataFrame()
ord_test['order_month'] = df_test['order_month'].astype('category')

In [778]:
### Encode categorical values
# 1. get variables that need to be 1-hot-encoded: item_size, item_color, user_title, user_state, order_month, order_year
# ... delivery_month, delivery_year, item_popularity, brand_popularity
cat_train = df_train[['item_size', 'item_color', 'user_title', 'user_state',
                     'delivery_month', 'delivery_year', 'item_popularity', 'brand_popularity']]

cat_test = df_train[['item_size', 'item_color', 'user_title', 'user_state',
                     'delivery_month', 'delivery_year', 'item_popularity', 'brand_popularity']]

ord_train = pd.DataFrame()
ord_train['order_month'] = df_train['order_month'].astype('category').copy()
ord_train['order_year'] = df_train['order_year'].astype('category').copy()

ord_test = pd.DataFrame()
ord_test['order_month'] = df_test['order_month'].astype('category').copy()
ord_test['order_year'] = df_test['order_year'].astype('category').copy()

cat_train['order_month'] = ord_train.order_month
cat_train['order_year'] = ord_train.order_year

cat_test['order_month'] = ord_test.order_month
cat_test['order_year'] = ord_test.order_year

cat_train = pd.get_dummies(cat_train)
cat_test = pd.get_dummies(cat_test)


C:\Users\Hagen\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hagen\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hagen\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [782]:
cat_train.columns

Index(['item_size_36', 'item_size_37', 'item_size_38', 'item_size_39',
       'item_size_40', 'item_size_41', 'item_size_42', 'item_size_43',
       'item_size_44', 'item_size_46', 'item_size_l', 'item_size_m',
       'item_size_other', 'item_size_s', 'item_size_unsized', 'item_size_xl',
       'item_size_xxl', 'item_color_anthracite', 'item_color_berry',
       'item_color_black', 'item_color_blue', 'item_color_brown',
       'item_color_denim', 'item_color_green', 'item_color_grey',
       'item_color_ocher', 'item_color_olive', 'item_color_other',
       'item_color_petrol', 'item_color_purple', 'item_color_red',
       'item_color_white', 'user_title_Company', 'user_title_Family',
       'user_title_Mr', 'user_title_Mrs', 'user_title_not reported',
       'user_state_Baden-Wuerttemberg', 'user_state_Bavaria',
       'user_state_Berlin', 'user_state_Brandenburg', 'user_state_Bremen',
       'user_state_Hamburg', 'user_state_Hesse', 'user_state_Lower Saxony',
       'user_state_Meckl

In [781]:
df_train.columns

Index(['item_size', 'item_color', 'item_price', 'user_title', 'user_state',
       'return', 'LOC', 'age', 'DT', 'npp', 'order_month', 'order_year',
       'delivery_month', 'delivery_year', 'item_popularity',
       'brand_popularity'],
      dtype='object')

In [802]:
# get continouus values
cont_train = df_train[['item_price', 'LOC', 'age', 'DT', 'npp']].copy()
cont_test = df_test[['item_price', 'LOC', 'age', 'DT', 'npp']].copy()

#standardize them
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(cont_train)
cont_train = pd.DataFrame(sc.transform(cont_train))
cont_test = pd.DataFrame(sc.transform(cont_test))


#join them back together
X_train = pd.concat([cat_train, cont_train], axis=1)
y_train = df_train['return']

X_test = pd.concat([cat_test, cont_test], axis=1)

In [808]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=2000.0, random_state=0)
lr.fit(X_train.values, y_train.values)
y_predict = lr.predict(X_train.values)
error = sum(abs(y_train.values - y_predict))
print('Error: {}'.format(error))

X_train.shape

Error: 37138


(100000, 94)

In [812]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(X_train.values, y_train.values)
y_predict = clf.predict(X_train.values)
error = sum(abs(y_train.values - y_predict))
print('Error: {}'.format(error))

X_train.shape

Error: 1426


(100000, 94)

In [813]:
1- (1426/100000)

0.98574